<a href="https://colab.research.google.com/github/edsondamasceno/classification-COVID-19/blob/main/CNN_Feature_Extraction_(Architecture_1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Library

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
from keras.models import Sequential, load_model, Model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from keras import optimizers
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Load Data

In [ ]:
train_path = '/content/drive/MyDrive/COVID-19/COVID-QU-Ex-V1/Train'
test_path = '/content/drive/MyDrive/COVID-19/COVID-QU-Ex-V1/Test'
val_path = '/content/drive/MyDrive/COVID-19/COVID-QU-Ex-V1/Val'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

train_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(train_path,
                                                                       target_size=(224,224),
                                                                       color_mode='rgb',
                                                                       batch_size=100,
                                                                       shuffle=True)
test_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(test_path,
                                                                      target_size=(224,224),
                                                                      color_mode='rgb',
                                                                      batch_size=100,
                                                                      shuffle=True)
val_batches = ImageDataGenerator(rescale=1./255).flow_from_directory(val_path,
                                                                     target_size=(224,224),
                                                                     color_mode='rgb',
                                                                     batch_size=100,
                                                                     shuffle=True)

# Create Model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras import optimizers

np.random.seed(1337)          # for reproducibility
print('Building model...')
model = Sequential()

#1st conv layer
model.add(Conv2D(64, (11,11), padding="same",
                 input_shape=(224, 224, 3),data_format="channels_last"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

#2nd conv layer
model.add(Conv2D(128, (7,7), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

#3rd conv layer
model.add(Conv2D(192, (5,5), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.4))

#4th conv layer
model.add(Conv2D(256, (3,3), padding="same"))
model.add(BatchNormalization())
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(128)
model.add(Dense(100,name ='feature_dense'))
#model.load_weights(by_name=True,filepath = filepath)

#output FC
model.add(Dense(2))
model.add(Activation('sigmoid'))
adam = optimizers.Adam(learning_rate=0.01, beta_1=0.9, beta_2=0.999)
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')
model.summary()

In [ ]:
history = model.fit_generator(train_batches,epochs=50,validation_data=val_batches,verbose=1)
model.save('/content/drive/MyDrive/COVID-19/Arquitetura-1/CNN_model.h5')

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({'font.size': 20})

val_accuracy = history.history['val_accuracy']
val_loss = history.history['val_loss']

accuracy = history.history['accuracy']
loss = history.history['loss']

epochs = range(1, len(accuracy) + 1)

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(epochs, val_accuracy, 'b', color='blue', label='Val Accuracy')
plt.plot(epochs, accuracy, 'b', color='red', label='Training Accuracy' )
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.savefig('/content/drive/MyDrive/COVID-19/Arquitetura-1/Acc_model.svg')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(epochs, accuracy, 'b', color='blue')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()

plt.savefig('/content/drive/MyDrive/COVID-19/Arquitetura-1/Val_Acc.svg')

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(epochs, val_loss, 'b',color='blue', label='Val Loss')
plt.plot(epochs, loss, 'b', color='red',label='Training Loss' )
plt.title('Loss CNN Model')
plt.legend()

plt.savefig('/content/drive/MyDrive/COVID-19/Arquitetura-1/loss_model.svg')

In [ ]:
import pandas as p
hist_df = pd.DataFrame(history.history)
hist_csv_file = '/content/drive/MyDrive/COVID-19/Arquitetura-1/history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

In [ ]:
results = model.evaluate(test_batches, verbose=0)

print("Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))

# Feature Extraction Methodology

In [ ]:
model = load_model('/content/drive/MyDrive/COVID-19/Arquitetura-1/CNN_model.h5')

In [ ]:
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('feature_dense').output)
intermediate_layer_model.summary()

In [ ]:
feauture_engg_data = intermediate_layer_model.predict(val_batches)
feauture_engg_data = pd.DataFrame(feauture_engg_data)
print('feauture_engg_data shape:', feauture_engg_data.shape)
feauture_engg_data.head(5)  #The features are unnamed now

In [ ]:
import numpy as np
import glob
import pandas as pd
import keras.utils as image

In [ ]:
def extract_feature_one_image(img_path,intermediate_layer_model,input_img):

    img = image.load_img(img_path, target_size=(input_img, input_img))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)

    features = intermediate_layer_model.predict(img_data)
    features = features.reshape((-1))
    return features

In [ ]:
def create_model():

    input_image = 224
    intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer('feature_dense').output)

    return intermediate_layer_model, input_image

In [ ]:
model_name = 'CNN'
database_name = 'Test'

path_database = '/content/drive/MyDrive/COVID-19/COVID-QU-Ex-V1/' + database_name + '/'
path_write = '/content/drive/MyDrive/COVID-19/Arquitetura-1/' + database_name + '_' + model_name + '.csv'

intermediate_layer_model, input_img = create_model()
features = []

print('Model created...')

classes = glob.glob(path_database + '*')
cont_classe = 0
features_labels_two_classes = pd.DataFrame()

for classe in classes:
    features = []
    images_names = glob.glob(classe + '/*.*')
    for name in images_names:
        features.append(extract_feature_one_image(name, intermediate_layer_model, input_img))

    features_labels = pd.DataFrame(data=features)
    features_labels['Labels'] = np.uint8(cont_classe)
    features_labels_two_classes = features_labels_two_classes.append(features_labels,
                                                                 ignore_index=True)
    cont_classe+=1
features_labels_two_classes.to_csv(path_write,index=False)

print('Finished!!!!')

In [ ]:
features_labels_two_classes.shape